In [7]:
from app.openai_sdk.seed_config import GENERATE_QUESTIONS_ASSISTANT_SEED, mock_generated_questions, mock_rag_content

In [8]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv


_: bool = load_dotenv(find_dotenv())  # read local .env file

client: OpenAI = OpenAI()



In [22]:
import json 

def generate_Questions(quiz_title: str, questions_to_generate: int, question_type:list, content: str):
    """
    Grade Open Text Question
    """
    try:
        seed_prompt = f"""Create a 10 minutes 'Generative AI' Quiz that will have `4` Questions. The Questions type will include: 1. "single_select_mcq" 2. "multi_select_mcq" and "open_text_question". The Quiz will be easy and the questions generated shall follow: {mock_rag_content}"""
        user_prompt = f"""Create a 30 minutes {quiz_title} Quiz that will have {questions_to_generate} Questions. The Questions type will include: {question_type}. The Quiz will be easy and the questions generated shall follow: {content}"""
        # 1. Call OpenAI API to grade the question
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": GENERATE_QUESTIONS_ASSISTANT_SEED},
                {"role": "user", "content": seed_prompt},
                {"role": "assistant", "content": json.dumps({'questions': mock_generated_questions})},
                {"role": "user", "content": user_prompt}
            ]
            )
        
        json_str = response.choices[0].message.content
        print(json_str)
        
        if json_str is None:
            raise ValueError("Question Grading Failed")
        
        # 2. Parse the response
        obj_out: dict[str, list[str]] = json.loads(json_str)

        return obj_out
    except Exception as e:
        raise ValueError(str(e))

In [ ]:
questions = generate_Questions("Generative AI", 4, ["single_select_mcq", "multi_select_mcq", "open_text_question"], mock_rag_content)

print("\n------------- questions\n", questions)